In [39]:
from preprocess import Preprocess
from urllib.request import urlopen
import timm
import torch
import functools
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [28]:
IMAGE_RESIZE = (64, 64)
TRANSFORM = transforms.Compose([
    transforms.Resize(IMAGE_RESIZE),
    transforms.ToTensor(),
])

In [32]:



class TIMM:
    def __init__(self):
        # https://huggingface.co/timm/vit_medium_patch16_reg4_gap_256.sbb_in1k
        self.model = timm.create_model(
            'vit_medium_patch16_reg4_gap_256.sbb_in1k',
            pretrained=True,
            num_classes=0,  # remove classifier nn.Linear
        )
        self.model.eval()

        # get model specific transforms (normalization, resize)
        data_config = timm.data.resolve_model_data_config(self.model)
        self.transforms = timm.data.create_transform(**data_config, is_training=False)

    @functools.lru_cache(maxsize=None)
    def extract(self, img) -> torch.Tensor:
        with torch.no_grad():
            return self.model(self.transforms(img).unsqueeze(0))[0]

timm_model = TIMM()

In [33]:
# transforma cada par em uma linha de uma ndarray (é quem criou os CSVs dev_train e dev_val)
def pairs_as_ndarray(pairs):
    result = []
    for i, pair in enumerate(pairs):
        image1, image2, label = pair
        print(f'Progress: {i}/{len(pairs)}', end='\r')
        attr1 = timm_model.extract(image1)
        attr2 = timm_model.extract(image2)
        row = np.concatenate((attr1, attr2, [label]))
        result.append(row)
    return np.array(result)

In [34]:
dev_train = pairs_as_ndarray(Preprocess.load_train_pairs(transform=TRANSFORM))
dev_val = pairs_as_ndarray(Preprocess.load_test_pairs(transform=TRANSFORM))
# dev_train = np.loadtxt('dev_train.csv', delimiter=',')
# dev_val = np.loadtxt('dev_val.csv', delimiter=',')
dev_train.shape, dev_val.shape

((2200, 1025), (1000, 1025))

In [35]:
# Save the arrays
np.save('dev_train.npy', dev_train)
np.save('dev_val.npy', dev_val)

In [37]:
X_train = dev_train[:, :-1]
y_train = dev_train[:, -1]
X_val = dev_val[:, :-1]
y_val = dev_val[:, -1]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((2200, 1024), (2200,), (1000, 1024), (1000,))

In [40]:
### Regressão logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': np.logspace(-5, 5, 5),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 0.0031622776601683794, 'solver': 'newton-cg'}
Best cross-validation score:  0.5622727272727273


In [41]:

logreg_model = LogisticRegression(**grid_search.best_params_)
logreg_model.fit(X_train, y_train)
print('Train accuracy:', accuracy_score(y_train, logreg_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, logreg_model.predict(X_val)))


Train accuracy: 0.7472727272727273
Val accuracy: 0.596


In [42]:

### Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=10, cv=5)
rf_random_search.fit(X_train, y_train)

print("Best parameters: ", rf_random_search.best_params_)
print("Best cross-validation score: ", rf_random_search.best_score_)


Best parameters:  {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20}
Best cross-validation score:  0.6568181818181819


In [43]:

rf_model = RandomForestClassifier(**rf_random_search.best_params_)
rf_model.fit(X_train, y_train)

print('Train accuracy:', accuracy_score(y_train, rf_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, rf_model.predict(X_val)))


Train accuracy: 1.0
Val accuracy: 0.687


In [44]:

### Rede neural
import keras_tuner as kt
import keras

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=4,
                                                max_value=32,
                                                step=4),
                                   activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=5,
    directory='runs',
    project_name=f'nn-timm-medium')

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=40, validation_data=(X_val, y_val))

tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(num_trials = 10)[0]


Trial 5 Complete [00h 00m 27s]
val_accuracy: 0.6579999923706055

Best val_accuracy So Far: 0.7248000025749206
Total elapsed time: 00h 02m 05s
Results summary
Results in runs\nn-timm-medium
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
num_layers: 1
units_0: 16
learning_rate: 0.001
units_1: 16
Score: 0.7248000025749206

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 12
learning_rate: 0.01
units_1: 32
Score: 0.7217999935150147

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 32
learning_rate: 0.001
units_1: 4
Score: 0.7197999954223633

Trial 2 summary
Hyperparameters:
num_layers: 2
units_0: 28
learning_rate: 0.01
units_1: 16
Score: 0.7194000005722045

Trial 4 summary
Hyperparameters:
num_layers: 3
units_0: 20
learning_rate: 0.0001
units_1: 8
units_2: 4
Score: 0.6579999923706055


In [45]:

best_hp = tuner.get_best_hyperparameters()[0]
nn_model = tuner.hypermodel.build(best_hp)
nn_model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
nn_model.summary()


Epoch 1/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5808 - loss: 0.7662 - val_accuracy: 0.6690 - val_loss: 0.6230
Epoch 2/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - accuracy: 0.7530 - loss: 0.5101 - val_accuracy: 0.6890 - val_loss: 0.5831
Epoch 3/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8324 - loss: 0.4067 - val_accuracy: 0.7040 - val_loss: 0.5544
Epoch 4/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - accuracy: 0.8631 - loss: 0.3506 - val_accuracy: 0.7130 - val_loss: 0.5649
Epoch 5/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - accuracy: 0.8998 - loss: 0.2890 - val_accuracy: 0.7130 - val_loss: 0.5762
Epoch 6/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - accuracy: 0.9329 - loss: 0.2338 - val_accuracy: 0.7100 - val_loss: 0.5975
Epoch 7/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - accuracy: 0.9598 - loss: 0.1890 - val_accuracy: 0.7080 - val_loss: 0.6168
Epoch 8/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - accuracy: 0.9657 - loss: 0.1621 - val_accuracy: 0.709

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 16)             │        16,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,253 (192.40 KB)

 Trainable params: 16,417 (64.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 32,836 (128.27 KB)

In [46]:

train_accuracy = nn_model.evaluate(X_train, y_train)[1]
val_accuracy = nn_model.evaluate(X_val, y_val)[1]
train_accuracy, val_accuracy


69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 1.0000 - loss: 0.0030
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.7275 - loss: 1.0108  


(1.0, 0.7089999914169312)

In [47]:


### SVM
from sklearn.svm import SVC

# Define the parameter grid for grid search
param_grid = {
    'C': np.logspace(-3, 2, base=10, num=6),
    'kernel': ['rbf', 'sigmoid'],
    'gamma': np.logspace(-3, 2, base=10, num=6)
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score:  0.7086363636363637


In [48]:

# Create an SVM model with the best parameters
svm_model = SVC(**grid_search.best_params_)
svm_model.fit(X_train, y_train)

# Evaluate the model
print('Train accuracy:', accuracy_score(y_train, svm_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, svm_model.predict(X_val)))

Train accuracy: 1.0
Val accuracy: 0.724
